In [2]:
# -----------------------------
# Imports
# -----------------------------
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

plt.style.use('seaborn-v0_8')

# -----------------------------
# Daten laden (Close-only)
# -----------------------------
TICKER = "ORCL" # Oracle Corporation
START = "2020-01-01" # Anfangsdatum
END   = "2025-01-01" # Enddatum

raw = yf.download( #  Daten von Yahoo Finance laden
    TICKER, # Ticker-Symbol
    start=START, # Anfangsdatum
    end=END, # Enddatum
    auto_adjust=False,# keine Anpassung für Dividenden/Stock Splits
    progress=False # kein Fortschrittsbalken anzeigen
)

# MultiIndex abfangen
if isinstance(raw.columns, pd.MultiIndex): # falls Yahoo Finance mehrere Ebenen liefert
    raw.columns = raw.columns.get_level_values(0) # nur die erste Ebene behalten

data = raw[['Close']].dropna().rename(columns={'Close': 'Kurs'}).copy() # Close in Kurs umbenennen

print("Anzahl Beobachtungen:", len(data)) # Ausgabe der Anzahl der Beobachtungen
print("Zeitraum:", data.index.min().date(), "bis", data.index.max().date()) # Ausgabe des Zeitraums
display(data.head()) # Anzeigen der ersten fünf Zeilen des DataFrames
display(data.tail()) # Anzeigen der letzten fünf Zeilen des DataFrame

# -----------------------------
# Deskriptive Statistik
# -----------------------------
stats = data['Kurs'].describe() # Berechnung der deskriptiven Statistik für die Kursdaten
print("Deskriptive Statistik zum Schlusskurs (Close):") # Ausgabe der deskriptiven Statistik
display(stats) # Anzeigen der deskriptiven Statistik

# -----------------------------
# Kursverlauf
# -----------------------------
plt.figure(figsize=(12, 5)) # Erstellen einer neuen Abbildung mit benutzerdefinierter Größe
plt.plot(data.index, data['Kurs'], label='Oracle (Close)') # Plotten des Kursverlaufs
plt.title('Oracle Corporation – Kursverlauf 2020–2025 (Close)')    # Titel des Plots
plt.xlabel('Datum')    # x-Achsenbeschriftung
plt.ylabel('Kurs in USD')  # y-Achsenbeschriftung
plt.legend() # Legende anzeigen
plt.grid(True, alpha=0.2) # Gitterlinien mit Transparenz
plt.tight_layout() # Layout anpassen
plt.show() # Plot anzeigen

# -----------------------------
# Tägliche Rendite + Kennzahlen
# -----------------------------
data['Rendite'] = data['Kurs'].pct_change() # Berechnung der täglichen Renditen / pct_change() braucht mindestens 2 Werte
returns = data['Rendite'].dropna() # Entfernen von NaN-Werten

mean_daily = returns.mean() # Durchschnittliche tägliche Rendite
vol_daily  = returns.std() #   Tägliche Volatilität (Standardabweichung)

print(f"Durchschnittliche tägliche Rendite: {mean_daily:.4%}") # Ausgabe der durchschnittlichen täglichen Rendite
print(f"Tägliche Volatilität (Standardabweichung): {vol_daily:.4%}") # Ausgabe der täglichen Volatilität

annual_return_est = (1 + mean_daily) ** 252 - 1 # Schätzung der jährlichen Rendite (annualisiert)
annual_vol_est    = vol_daily * np.sqrt(252) # Schätzung der jährlichen Volatilität (annualisiert)

print(f"Geschätzte jährliche Rendite (annualisiert): {annual_return_est:.2%}") #   Ausgabe der geschätzten jährlichen Rendite
print(f"Geschätzte jährliche Volatilität (annualisiert): {annual_vol_est:.2%}") # Ausgabe der geschätzten jährlichen Volatilität

# -----------------------------
# Histogramm der täglichen Renditen
# -----------------------------
plt.figure(figsize=(10, 5)) # Erstellen einer neuen Abbildung
plt.hist(returns, bins=50) # Histogramm der täglichen Renditen mit 50 Bins
plt.title('Verteilung der täglichen Renditen – Oracle (Close)')  # Titel desPlots
plt.xlabel('Tägliche Rendite') # x-Achsenbeschriftung
plt.ylabel('Häufigkeit') # y-Achsenbeschriftung
plt.grid(True, alpha=0.2) # Gitterlinien mit Transparenz
plt.tight_layout()
plt.show()

# -----------------------------
# Kumulative Rendite
# -----------------------------
data['Kumulierte_Rendite'] = (1 + data['Rendite'].fillna(0)).cumprod() - 1 # Berechnung der kumulativen Rendite

plt.figure(figsize=(12, 5)) # Erstellen einer neuen Abbildung
plt.plot(data.index, data['Kumulierte_Rendite'], label='Kumulierte Rendite') # Plotten der kumulativen Rendite
plt.title('Oracle – Kumulative Rendite seit 2020 (Close-basiert)') # Titel des Plots
plt.xlabel('Datum') # x-Achsenbeschriftung
plt.ylabel('Kumulative Rendite') # y-Achsenbeschriftung
plt.legend() # Legende anzeigen
plt.grid(True, alpha=0.2) # Gitterlinien mit Transparenz
plt.tight_layout() # Layout anpassen
plt.show() # Plot anzeigen

# -----------------------------
# Drawdown (auf Basis kumulativer Rendite)
# -----------------------------
cum = (1 + returns).cumprod() # kumulierte Rendite
running_max = cum.cummax() # bisheriges Maximum der kumulierten Rendite
drawdown = cum / running_max - 1 # Drawdown berechnen - Differenz zur bisherigen Höchstmarke - (aktueller Wert / bisheriges Maximum) - 1

plt.figure(figsize=(12, 4)) # Erstellen einer neuen Abbildung
plt.plot(cum.index, drawdown, color='red') # Plotten des Drawdowns
plt.title('Drawdown (kumulierte Rendite vs. bisheriges Maximum)')
plt.ylabel('Drawdown')
plt.grid(True, alpha=0.2) # Gitterlinien mit Transparenz
plt.tight_layout() # Layout anpassen
plt.show()

print(f"Max Drawdown: {drawdown.min():.2%} am {drawdown.idxmin().date()}") # Ausgabe des maximalen Drawdowns und des Datums

# =========================================================
#  Jahresrenditen (YoY) – Balkendiagramm
# =========================================================
year_end_close = data['Kurs'].resample('YE').last() # Jahresend-Kurse (Year-End Close)
annual_returns = year_end_close.pct_change().dropna() # Jährliche Renditen berechnen
annual_returns.index = annual_returns.index.year # Index auf Jahre setzen

plt.figure(figsize=(10, 5)) # Erstellen einer neuen Abbildung
plt.bar(annual_returns.index.astype(str), annual_returns.values * 100, color=['green' if v >= 0 else 'red' for v in annual_returns.values]) # Balkendiagramm der jährlichen Renditen
plt.title('Oracle – Jährliche Renditen (YoY, Close)') # Titel des Plots
plt.xlabel('Jahr') # x-Achsenbeschriftung
plt.ylabel('Rendite in %')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter()) # y-Achse als Prozent formatieren
plt.grid(True, axis='y', alpha=0.2) # Gitterlinien auf y-Achse mit Transparenz
plt.tight_layout() # Layout anpassen
plt.show()

# =========================================================
#  Rollierende 30-Tage-Volatilität (annualisiert)
# =========================================================
vol_30d_ann = returns.rolling(30).std() * np.sqrt(252)

plt.figure(figsize=(12, 5))
plt.plot(vol_30d_ann.index, vol_30d_ann.values * 100) # in Prozent umrechnen und plotten
plt.title('Oracle – Rollierende 30-Tage-Volatilität (annualisiert, Close)')
plt.xlabel('Datum')
plt.ylabel('Volatilität in %')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter()) # y-Achse als Prozent formatieren
plt.grid(True, alpha=0.2)
plt.tight_layout()
plt.show()

# =========================================================
# Dividenden ab 2020 + Dividendenrendite pro Jahr
# =========================================================
ORCL = yf.Ticker(TICKER) # Ticker-Objekt für Oracle erstellen
dividends = orcl.dividends.loc["2020-01-01":] # Dividenden ab 2020 filtern

# jährliche Dividendensumme
annual_dividends = dividends.resample("YE").sum().dropna() # jährliche Dividenden summieren
annual_dividends.index = annual_dividends.index.year # Index auf Jahre setzen

plt.figure(figsize=(10, 5))
plt.bar(annual_dividends.index.astype(str), annual_dividends.values) # Balkendiagramm der jährlichen Dividenden
plt.title("Oracle – Jährliche Dividendenauszahlung je Aktie (ab 2020)") # Titel des Plots
plt.xlabel("Jahr")
plt.ylabel("Dividende je Aktie (USD)")
plt.grid(True, axis="y", alpha=0.2)
plt.tight_layout()
plt.show()

# Dividendenrendite pro Jahr: Dividenden / Year-End Close (Close-basiert)
year_end_close_year = year_end_close.copy() # Jahresend-Close-Kurse kopieren
year_end_close_year.index = year_end_close_year.index.year # Index auf Jahre setzen

dividend_yield = (annual_dividends / year_end_close_year).dropna() # Dividendenrendite berechnen

plt.figure(figsize=(10, 5))
plt.plot(dividend_yield.index.astype(str), dividend_yield.values * 100, marker="o") # in Prozent umrechnen und plotten
plt.title("Oracle – Dividendenrendite pro Jahr (Dividende / Jahresend-Close)") # Titel des Plots
plt.xlabel("Jahr") # x-Achsenbeschriftung
plt.ylabel("Dividendenrendite in %") # y-Achsenbeschriftung
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter()) # y-Achse als Prozent formatieren
plt.grid(True, axis="y", alpha=0.2)
plt.tight_layout()
plt.show()

# ---------------------------------------------------------
# Übersicht Jahresrenditen, Volatilität, Dividenden
# ---------------------------------------------------------
overview = pd.DataFrame({ # Erstellung eines DataFrames für die Jahresübersicht
    "Jahresrendite (YoY)": (annual_returns * 100).round(2), # in Prozent umrechnen und runden
    "Volatilität (ann., pro Jahr)": ( # annualisierte Volatilität pro Jahr
        returns.resample("YE").std() * np.sqrt(252) # annualisierte Volatilität berechnen
    ).dropna().set_axis( # Jahre als Index setzen
        (returns.resample("YE").std() * np.sqrt(252)).dropna().index.year # Jahre extrahieren
    ).round(2), # runden
    "Dividende je Aktie (Jahr)": annual_dividends.round(2), # Dividende je Aktie runden
    "Dividendenrendite (Jahr)": (dividend_yield * 100).round(2) # Dividendenrendite in Prozent runden
}).sort_index() # nach Index sortieren

print("\n--- Jahresübersicht (ab 2020) ---") # Ausgabe der Jahresübersicht
print(overview.to_string()) # Anzeige der Jahresübersicht als String

ModuleNotFoundError: No module named 'numpy'